#### Importation des Bibliotheques

In [1]:
import pandas as pd
import os
from google.cloud import bigquery
from pandas_gbq import to_gbq, read_gbq

#### Reccuepration des parametres

In [2]:
os.chdir("/PARIS_OPENDATA_GCP/Workspace/Velib")

%run ../Commun/Params.ipynb

In [3]:
print(parametres)

{'BQ_BRONZE_DATASET': 'paris-opendata-gcp.bronze', 'BQ_GLOD_DATASET': 'paris-opendata-gcp.gold', 'BQ_SILVER_DATASET': 'paris-opendata-gcp.silver', 'BUCKET_NAME': 'paris-opendata-gcp-data', 'REGION': 'region', 'TEMP_GCS_BUCKET': 'paris-opendata-gcp-data/temporary', 'URL_API_VELIB': 'https://opendata.paris.fr/api/records/1.0/search/?dataset=velib-disponibilites-en-temps-reel&q=&rows=10000&facet=contract_name&facet=station_state&facet=banking&facet=bonus&facet=last_update'}


In [4]:
BQ_BRONZE_DATASET =  parametres["BQ_BRONZE_DATASET"]
SILVER_BRONZE_DATASET =  parametres["BQ_SILVER_DATASET"]

TEMP_GCS_BUCKET = parametres["TEMP_GCS_BUCKET"]
BUCKET_NAME = parametres["BUCKET_NAME"]
BQ_BRONZE_TABLE = f"{BQ_BRONZE_DATASET}.velib_emplacement_stations"
BQ_SILVER_TABLE = f"{SILVER_BRONZE_DATASET}.velib_emplacement_stations"

##### Lecture du fichier csv Velib emplacement des stations

In [5]:
bronze_station_velib_df = pd.read_gbq(BQ_BRONZE_TABLE)

##### Creation des champs latitude et longitute

In [6]:
bronze_station_velib_df[["Longitude", "Latitude"]] = bronze_station_velib_df["Coordonnées géographiques"].str.split(",", expand=True).astype(float)

In [7]:
df_final = bronze_station_velib_df.drop("Coordonnées géographiques", axis =1)

#### Charegement de la table Bronze

In [8]:
to_gbq(
    dataframe=df_final,
    destination_table=f"{BQ_SILVER_TABLE}",
    if_exists="replace"
)

100%|██████████| 1/1 [00:00<00:00, 2626.36it/s]
